In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# Read the 'Netflix_Engagement_Plus.csv'data file as Netflix_df
Netflix_df = pd.read_excel("Resources/Netflix_Engagement_Report.xlsx", header=5)
Netflix_df = Netflix_df.drop(['Unnamed: 0'], axis=1)
Netflix_df

,Title,Available Globally?,Release Date,Hours Viewed
0,The Night Agent: Season 1,Yes,2023-03-23,812100000
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000
2,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30,622800000
3,Wednesday: Season 1,Yes,2022-11-23,507700000
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000
...,...,...,...,...
18209,راس السنة,No,NaT,100000
18210,心が叫びたがってるんだ。,No,NaT,100000
18211,두근두근 내 인생,No,NaT,100000
18212,라디오 스타,No,NaT,100000


In [3]:
# Clean up Title name and split off OriginalTitle and Year_Country
Netflix2_df = Netflix_df
Netflix2_df['originalTitle'] = Netflix2_df['Title'].str.extract('(?:\/\/)(.+(?=:))')
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\/\/.{1,}', '', regex=True)
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\:\sSeason\s\d+', '', regex=True)
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\:\sPart\s\d+', '', regex=True)
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\:\sBook\s\d+', '', regex=True)
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\:\sVolume\s\d+', '', regex=True)
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\:\sLimited\sSeries', '', regex=True)
Netflix2_df['Year_Country'] = Netflix2_df['Title'].str.extract('(?:\()(.+(?=\)))')
Netflix2_df['Title'] = Netflix2_df['Title'].str.replace('\(.+', '', regex=True)
Netflix2_df['Title'] = Netflix2_df['Title'].str.strip()
Netflix2_df['originalTitle'] = Netflix2_df['originalTitle'].str.strip()
Netflix2_df.head()

,Title,Available Globally?,Release Date,Hours Viewed,originalTitle,Year_Country
0,The Night Agent,Yes,2023-03-23,812100000,NaN,NaN
1,Ginny & Georgia,Yes,2023-01-05,665100000,NaN,NaN
2,The Glory,Yes,2022-12-30,622800000,더 글로리,NaN
3,Wednesday,Yes,2022-11-23,507700000,NaN,NaN
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000,NaN,NaN


In [4]:
# Merge, and clean up the Netflix data so that all seasons have their hours viewed combined
df = Netflix2_df[['Title','Hours Viewed']]
df = pd.DataFrame(df.groupby(by='Title').sum())
df = pd.merge(df,Netflix2_df, on='Title', how='left')
df = df.sort_values(by=['Release Date'])
df = df.drop_duplicates(subset=['Title'])
df = df.drop(columns=['Hours Viewed_y'])
df = df.rename(columns={'Hours Viewed_x':'Hours Viewed'})
df['Title'] = df['Title'].str.lower()
df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country
1164,arrested development,41300000,No,2010-04-01,NaN,NaN
16685,trailer park boys,78100000,Yes,2010-09-22,NaN,NaN
7904,la reina del sur,616800000,No,2011-09-05,NaN,NaN
8253,lilyhammer,7800000,No,2012-02-06,NaN,NaN
6400,house of cards,68900000,No,2013-02-01,NaN,NaN
...,...,...,...,...,...,...
18209,레드슈즈,200000,No,NaT,NaN,NaN
18210,비상선언,18600000,No,NaT,NaN,NaN
18211,선생 김봉두,100000,No,NaT,NaN,NaN
18212,침묵,400000,No,NaT,NaN,NaN


In [5]:
# Sort dataframe by hours viewed to review most watched shows after season data has been combined
df = df.sort_values(by=['Hours Viewed'], ascending=False)
df.head(50)

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country
5379,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN
15439,the night agent,812100000,Yes,2023-03-23,NaN,NaN
17960,you,766300000,Yes,2018-12-26,NaN,NaN
10764,outer banks,740400000,Yes,2020-04-15,NaN,NaN
16178,the walking dead,738600000,No,NaT,NaN,NaN
14713,the glory,622800000,Yes,2022-12-30,더 글로리,NaN
7904,la reina del sur,616800000,No,2011-09-05,NaN,NaN
2999,cocomelon,601200000,No,2020-05-31,NaN,NaN
13394,suits,599300000,Yes,NaT,NaN,2011
14084,the blacklist,596900000,No,NaT,NaN,NaN


In [6]:
# Extracting the year from 'Release Date' to new column called 'release_year'
df['release_year'] = df['Release Date'].dt.strftime('%Y')
df.head()

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year
5379,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN,2021
15439,the night agent,812100000,Yes,2023-03-23,NaN,NaN,2023
17960,you,766300000,Yes,2018-12-26,NaN,NaN,2018
10764,outer banks,740400000,Yes,2020-04-15,NaN,NaN,2020
16178,the walking dead,738600000,No,NaT,NaN,NaN,NaN


In [7]:
# Read the 'Netflix_Engagement_Plus.csv'data file as engagement_df
title_df = pd.read_csv("Resources/data_title.tsv",sep = '\t') 
title_df.head()

C:\Users\Joey\AppData\Local\Temp\ipykernel_9288\146890606.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_df = pd.read_csv("Resources/data_title.tsv",sep = '\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
# Read the 'data_ratings.tsv'data file as score_df
score_df = pd.read_csv("Resources/data_ratings.tsv",sep = '\t') 
score_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2008
1,tt0000002,5.7,270
2,tt0000003,6.5,1926
3,tt0000004,5.4,178
4,tt0000005,6.2,2701


In [9]:
# Check what titleTypes IMDB's database has
title_df['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [10]:
# Eliminate titleTypes which are not needed for Netflix data matching and format IMDB title information to match Netflix Data
Filtered_title_df = title_df[~title_df['titleType'].isin(['videoGame', 'tvPilot', 'tvEpisode', 'short', 'video', 'tvMovie', 'tvShort'])]
Filtered_title_df = Filtered_title_df.rename(columns={'primaryTitle':'Title'})
Filtered_title_df['Title'] = Filtered_title_df['Title'].str.lower()
Filtered_title_df['originalTitle'] = Filtered_title_df['originalTitle'].str.lower()
Filtered_title_df

,tconst,titleType,Title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,miss jerry,miss jerry,0,1894,\N,45,Romance
144,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897,\N,100,"Documentary,News,Sport"
498,tt0000502,movie,bohemios,bohemios,0,1905,\N,100,\N
570,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907,\N,90,Drama
...,...,...,...,...,...,...,...,...,...
10415815,tt9916678,tvSeries,acelerados,acelerados,0,2019,\N,\N,Comedy
10415816,tt9916680,movie,de la ilusión al desconcierto: cine colombiano...,de la ilusión al desconcierto: cine colombiano...,0,2007,\N,100,Documentary
10415828,tt9916706,movie,dankyavar danka,dankyavar danka,0,2013,\N,\N,Comedy
10415838,tt9916730,movie,6 gunn,6 gunn,0,2017,\N,116,Drama


In [11]:
# Merge Netflix and IMDB information
Merged_df = pd.merge(df,Filtered_title_df, on='Title', how='left')
Merged_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle_x,Year_Country,release_year,tconst,titleType,originalTitle_y,isAdult,startYear,endYear,runtimeMinutes,genres
0,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN,2021,tt10813940,tvSeries,ginny & georgia,0,2021,\N,\N,"Comedy,Drama"
1,the night agent,812100000,Yes,2023-03-23,NaN,NaN,2023,tt13918776,tvSeries,the night agent,0,2023,\N,45,"Action,Drama,Thriller"
2,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt0134224,movie,you,0,1968,\N,\N,Drama
3,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt0960886,movie,you,0,2009,\N,84,Drama
4,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt1487831,tvSeries,you,0,1982,1987,60,Talk-Show
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36740,endless night,100000,No,NaT,NaN,NaN,NaN,tt6605564,movie,endless night,0,1972,\N,86,"Horror,Thriller"
36741,enemy at the gates,100000,No,NaT,NaN,NaN,NaN,tt0215750,movie,enemy at the gates,0,2001,\N,131,"Action,Drama,War"
36742,area 51,100000,No,NaT,NaN,NaN,NaN,tt1519461,movie,area 51,0,2015,\N,91,"Horror,Mystery,Sci-Fi"
36743,my god!! father,100000,No,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Attempt 1: Match IMDB data to Netflix based on titles alone
Merged_df['Duplicated_values'] = Merged_df['Title'].duplicated(keep=False)
Title_matching_df = Merged_df
Title_matching_df = Title_matching_df[Title_matching_df['Duplicated_values'] == False]
Title_matching_df = Title_matching_df[Title_matching_df['tconst'].notnull()]
Title_matching_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle_x,Year_Country,release_year,tconst,titleType,originalTitle_y,isAdult,startYear,endYear,runtimeMinutes,genres,Duplicated_values
0,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN,2021,tt10813940,tvSeries,ginny & georgia,0,2021,\N,\N,"Comedy,Drama",False
1,the night agent,812100000,Yes,2023-03-23,NaN,NaN,2023,tt13918776,tvSeries,the night agent,0,2023,\N,45,"Action,Drama,Thriller",False
10,outer banks,740400000,Yes,2020-04-15,NaN,NaN,2020,tt10293938,tvSeries,outer banks,0,2020,\N,50,"Action,Crime,Drama",False
18,cocomelon,601200000,No,2020-05-31,NaN,NaN,2020,tt12427840,tvSeries,cocomelon,0,2018,2022,4,"Animation,Family",False
27,grey's anatomy,560300000,No,NaT,NaN,NaN,NaN,tt0413573,tvSeries,grey's anatomy,0,2005,\N,41,"Drama,Romance",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36730,terror in resonance,100000,No,NaT,残響のテロル,NaN,NaN,tt3613454,tvMiniSeries,zankyô no teroru,0,2014,2014,23,"Animation,Crime,Drama",False
36731,arianna,100000,No,NaT,NaN,NaN,NaN,tt4908430,movie,arianna,0,2015,\N,84,"Drama,Mystery",False
36741,enemy at the gates,100000,No,NaT,NaN,NaN,NaN,tt0215750,movie,enemy at the gates,0,2001,\N,131,"Action,Drama,War",False
36742,area 51,100000,No,NaT,NaN,NaN,NaN,tt1519461,movie,area 51,0,2015,\N,91,"Horror,Mystery,Sci-Fi",False


In [13]:
# Format data down to only what is needed for the purpose of analysis
Title_matching_df = Title_matching_df.filter(items=['Title', 'Hours Viewed','Available Globally?','Release Date',
                                                'originalTitle_x', 'Year_Country', 'release_year', 'tconst', 
                                                'titleType', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
Title_matching_df = Title_matching_df.rename(columns={'originalTitle_x':'originalTitle'})

In [14]:
# Merge title matching data with score data, and remove titles which did not pull any score information from their related IMDB entry
Title_matching_df = pd.merge(Title_matching_df,score_df, left_on='tconst', right_on='tconst', how='left')
Title_matching_df = Title_matching_df[Title_matching_df['averageRating'].notnull()]
Title_matching_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year,tconst,titleType,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN,2021,tt10813940,tvSeries,2021,\N,\N,"Comedy,Drama",7.5,77525.0
1,the night agent,812100000,Yes,2023-03-23,NaN,NaN,2023,tt13918776,tvSeries,2023,\N,45,"Action,Drama,Thriller",7.5,99387.0
2,outer banks,740400000,Yes,2020-04-15,NaN,NaN,2020,tt10293938,tvSeries,2020,\N,50,"Action,Crime,Drama",7.5,74515.0
3,cocomelon,601200000,No,2020-05-31,NaN,NaN,2020,tt12427840,tvSeries,2018,2022,4,"Animation,Family",4.3,895.0
4,grey's anatomy,560300000,No,NaT,NaN,NaN,NaN,tt0413573,tvSeries,2005,\N,41,"Drama,Romance",7.6,333561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430,terror in resonance,100000,No,NaT,残響のテロル,NaN,NaN,tt3613454,tvMiniSeries,2014,2014,23,"Animation,Crime,Drama",7.8,13133.0
7431,arianna,100000,No,NaT,NaN,NaN,NaN,tt4908430,movie,2015,\N,84,"Drama,Mystery",6.2,720.0
7432,enemy at the gates,100000,No,NaT,NaN,NaN,NaN,tt0215750,movie,2001,\N,131,"Action,Drama,War",7.5,274251.0
7433,area 51,100000,No,NaT,NaN,NaN,NaN,tt1519461,movie,2015,\N,91,"Horror,Mystery,Sci-Fi",4.3,14994.0


In [15]:
# Attempt 2: Match years, check for titles that have the same 'startYear' and 'release_year'
Year_matching_df = Merged_df[Merged_df['Duplicated_values'] == True]
Year_matching_df = Year_matching_df[Year_matching_df['startYear']==Year_matching_df['release_year']]
Year_matching_df['Duplicated_values'] = Year_matching_df['Title'].duplicated(keep=False)
Year_matching_df = Year_matching_df[Year_matching_df['Duplicated_values'] == False]
Year_matching_df = Year_matching_df[Year_matching_df['tconst'].notnull()]
Year_matching_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle_x,Year_Country,release_year,tconst,titleType,originalTitle_y,isAdult,startYear,endYear,runtimeMinutes,genres,Duplicated_values
14,the glory,622800000,Yes,2022-12-30,더 글로리,NaN,2022,tt21344706,tvSeries,deo geullori,0,2022,\N,50,"Drama,Mystery,Thriller",False
29,wednesday,507700000,Yes,2022-11-23,NaN,NaN,2022,tt13443470,tvSeries,wednesday,0,2022,\N,45,"Comedy,Crime,Fantasy",False
63,lucifer,434300000,No,2017-07-23,NaN,NaN,2017,tt5096524,movie,lucifer,0,2017,\N,110,Horror,False
94,never have i ever,341300000,Yes,2020-04-27,NaN,NaN,2020,tt10062292,tvSeries,never have i ever,0,2020,2023,30,"Comedy,Drama",False
118,sex/life,301900000,Yes,2021-06-25,NaN,NaN,2021,tt10839422,tvSeries,sex/life,0,2021,2023,45,"Comedy,Drama,Romance",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31622,mercury 13,100000,Yes,2018-04-20,NaN,NaN,2018,tt8139850,movie,mercury 13,0,2018,\N,78,Documentary,False
32191,whispers,100000,Yes,2020-06-11,وساوس,NaN,2020,tt12410506,tvSeries,whispers,0,2020,2020,\N,"Mystery,Thriller",False
32244,slam,100000,Yes,2017-04-15,NaN,NaN,2017,tt7503744,tvSeries,slam,0,2017,2018,\N,Game-Show,False
32298,strong island,100000,Yes,2017-09-15,NaN,NaN,2017,tt5873150,movie,strong island,0,2017,\N,107,"Crime,Documentary",False


In [16]:
# Format data down to only what is needed for the purpose of analysis
Year_matching_df = Year_matching_df.filter(items=['Title', 'Hours Viewed','Available Globally?','Release Date',
                                                'originalTitle_x', 'Year_Country', 'release_year', 'tconst', 
                                                'titleType', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
Year_matching_df = Year_matching_df.rename(columns={'originalTitle_x':'originalTitle'})
Year_matching_df.head()

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year,tconst,titleType,startYear,endYear,runtimeMinutes,genres
14,the glory,622800000,Yes,2022-12-30,더 글로리,NaN,2022,tt21344706,tvSeries,2022,\N,50,"Drama,Mystery,Thriller"
29,wednesday,507700000,Yes,2022-11-23,NaN,NaN,2022,tt13443470,tvSeries,2022,\N,45,"Comedy,Crime,Fantasy"
63,lucifer,434300000,No,2017-07-23,NaN,NaN,2017,tt5096524,movie,2017,\N,110,Horror
94,never have i ever,341300000,Yes,2020-04-27,NaN,NaN,2020,tt10062292,tvSeries,2020,2023,30,"Comedy,Drama"
118,sex/life,301900000,Yes,2021-06-25,NaN,NaN,2021,tt10839422,tvSeries,2021,2023,45,"Comedy,Drama,Romance"


In [17]:
# Merge year matching data with score data, and remove titles which did not pull any score information from their related IMDB entry
Year_matching_df = pd.merge(Year_matching_df,score_df, left_on='tconst', right_on='tconst', how='left')
Year_matching_df = Year_matching_df[Year_matching_df['averageRating'].notnull()]
Year_matching_df.head()

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year,tconst,titleType,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,the glory,622800000,Yes,2022-12-30,더 글로리,NaN,2022,tt21344706,tvSeries,2022,\N,50,"Drama,Mystery,Thriller",8.1,23866.0
1,wednesday,507700000,Yes,2022-11-23,NaN,NaN,2022,tt13443470,tvSeries,2022,\N,45,"Comedy,Crime,Fantasy",8.1,342374.0
2,lucifer,434300000,No,2017-07-23,NaN,NaN,2017,tt5096524,movie,2017,\N,110,Horror,6.1,11.0
3,never have i ever,341300000,Yes,2020-04-27,NaN,NaN,2020,tt10062292,tvSeries,2020,2023,30,"Comedy,Drama",7.9,76801.0
4,sex/life,301900000,Yes,2021-06-25,NaN,NaN,2021,tt10839422,tvSeries,2021,2023,45,"Comedy,Drama,Romance",5.6,29864.0


In [18]:
# Merge Title and Year matching data into one DF
Complete_data_df = pd.concat([Title_matching_df, Year_matching_df], ignore_index=True, sort=False)
Complete_data_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year,tconst,titleType,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN,2021,tt10813940,tvSeries,2021,\N,\N,"Comedy,Drama",7.5,77525.0
1,the night agent,812100000,Yes,2023-03-23,NaN,NaN,2023,tt13918776,tvSeries,2023,\N,45,"Action,Drama,Thriller",7.5,99387.0
2,outer banks,740400000,Yes,2020-04-15,NaN,NaN,2020,tt10293938,tvSeries,2020,\N,50,"Action,Crime,Drama",7.5,74515.0
3,cocomelon,601200000,No,2020-05-31,NaN,NaN,2020,tt12427840,tvSeries,2018,2022,4,"Animation,Family",4.3,895.0
4,grey's anatomy,560300000,No,NaT,NaN,NaN,NaN,tt0413573,tvSeries,2005,\N,41,"Drama,Romance",7.6,333561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8014,seth rogen's hilarity for charity,100000,Yes,2018-04-06,NaN,NaN,2018,tt8178116,tvSpecial,2018,\N,70,Comedy,5.0,2486.0
8015,mercury 13,100000,Yes,2018-04-20,NaN,NaN,2018,tt8139850,movie,2018,\N,78,Documentary,7.0,1273.0
8016,whispers,100000,Yes,2020-06-11,وساوس,NaN,2020,tt12410506,tvSeries,2020,2020,\N,"Mystery,Thriller",5.5,391.0
8017,strong island,100000,Yes,2017-09-15,NaN,NaN,2017,tt5873150,movie,2017,\N,107,"Crime,Documentary",6.4,4071.0


In [19]:
# Re-merge and compare completed data with original Netflix Dataframe to find missing titles
Missing_data_df = pd.concat([df, Complete_data_df], ignore_index=False, sort=False)
Missing_data_df['Duplicated_values'] = Missing_data_df['Title'].duplicated(keep=False)
Missing_data_df = Missing_data_df[Missing_data_df['Duplicated_values'] == False]
Missing_data_df = Missing_data_df.filter(items=['Title', 'Hours Viewed','Available Globally?','Release Date', 'originalTitle', 'Year_Country', 'release_year'])
Missing_data_df = Missing_data_df.sort_values(by=['Hours Viewed'], ascending=False)
Missing_data_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year
17960,you,766300000,Yes,2018-12-26,NaN,NaN,2018
16178,the walking dead,738600000,No,NaT,NaN,NaN,NaN
7904,la reina del sur,616800000,No,2011-09-05,NaN,NaN,2011
13394,suits,599300000,Yes,NaT,NaN,2011,NaN
14084,the blacklist,596900000,No,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16348,they are everywhere,100000,No,NaT,NaN,NaN,NaN
16345,there's a reason for the love i'm wearing,100000,No,NaT,着飾る恋には理由があって,NaN,NaN
7107,jackie chan's project a,100000,No,NaT,NaN,NaN,NaN
8063,le fils du français,100000,No,NaT,NaN,NaN,NaN


In [20]:
# Attempt 3: Merge with original title data from IMDB and title data from Netflix
Original_title_matching_df = pd.merge(Missing_data_df,Filtered_title_df, left_on='Title', right_on='originalTitle', how='left')
Original_title_matching_df

,Title_x,Hours Viewed,Available Globally?,Release Date,originalTitle_x,Year_Country,release_year,tconst,titleType,Title_y,originalTitle_y,isAdult,startYear,endYear,runtimeMinutes,genres
0,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt0134224,movie,you,you,0,1968,\N,\N,Drama
1,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt0960886,movie,you,you,0,2009,\N,84,Drama
2,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt1487831,tvSeries,you,you,0,1982,1987,60,Talk-Show
3,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt20123090,movie,you,you,0,2013,\N,326,Drama
4,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt7025370,movie,you,you,0,2018,\N,48,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21763,they are everywhere,100000,No,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21764,there's a reason for the love i'm wearing,100000,No,NaT,着飾る恋には理由があって,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21765,jackie chan's project a,100000,No,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21766,le fils du français,100000,No,NaT,NaN,NaN,NaN,tt0211372,movie,the son of français,le fils du français,0,1999,\N,107,"Adventure,Comedy,Family"


In [21]:
# Merge original title data with score data and sort for most voted entries, then keep only the most voted entry to elimniate extraneous information
Original_title_matching_df = pd.merge(Original_title_matching_df,score_df, left_on='tconst', right_on='tconst', how='left')
Original_title_matching_df = Original_title_matching_df.sort_values(by=['numVotes'], ascending=False)
Original_title_matching_df = Original_title_matching_df.drop_duplicates(subset=['Title_x'])
Original_title_matching_df = Original_title_matching_df[Original_title_matching_df['tconst'].notnull()]
Original_title_matching_df

,Title_x,Hours Viewed,Available Globally?,Release Date,originalTitle_x,Year_Country,release_year,tconst,titleType,Title_y,originalTitle_y,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
1277,the dark knight,18300000,No,NaT,NaN,2008,NaN,tt0468569,movie,the dark knight,the dark knight,0,2008,\N,152,"Action,Crime,Drama",9.0,2812745.0
2859,inception,7400000,No,NaT,NaN,NaN,NaN,tt1375666,movie,inception,inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,2497034.0
1515,the godfather,15400000,No,NaT,NaN,NaN,NaN,tt0068646,movie,the godfather,the godfather,0,1972,\N,175,"Crime,Drama",9.2,1973440.0
1586,gladiator,14900000,No,NaT,NaN,2000,NaN,tt0172495,movie,gladiator,gladiator,0,2000,\N,155,"Action,Adventure,Drama",8.5,1584096.0
2412,the wolf of wall street,9100000,No,NaT,NaN,NaN,NaN,tt0993846,movie,the wolf of wall street,the wolf of wall street,0,2013,\N,180,"Biography,Comedy,Crime",8.2,1536866.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21573,the rope,100000,No,NaT,NaN,NaN,NaN,tt15127510,tvSeries,the rope,the rope,0,\N,\N,\N,Drama,NaN,NaN
21613,thomas & friends: marvelous machinery: a new a...,100000,No,NaT,NaN,NaN,NaN,tt14594718,movie,thomas & friends: marvelous machinery: a new a...,thomas & friends: marvelous machinery: a new a...,0,2020,\N,\N,Animation,NaN,NaN
21629,jail breakers,100000,No,NaT,NaN,NaN,NaN,tt28660225,movie,jail breakers,jail breakers,0,2023,\N,51,Comedy,NaN,NaN
21660,zion,100000,Yes,2018-08-10,NaN,NaN,2018,tt30330486,movie,zion,zion,0,\N,\N,\N,\N,NaN,NaN


In [22]:
# Format data down to only what is needed for the purpose of analysis
Original_title_matching_df = Original_title_matching_df.filter(items=['Title_x', 'Hours Viewed','Available Globally?','Release Date',
                                                'originalTitle_x', 'Year_Country', 'release_year', 'tconst', 
                                                'titleType', 'startYear', 'endYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes'])
Original_title_matching_df = Original_title_matching_df.rename(columns={'originalTitle_x':'originalTitle', 'Title_x':'Title'})
Original_title_matching_df.head()

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year,tconst,titleType,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
1277,the dark knight,18300000,No,NaT,NaN,2008,NaN,tt0468569,movie,2008,\N,152,"Action,Crime,Drama",9.0,2812745.0
2859,inception,7400000,No,NaT,NaN,NaN,NaN,tt1375666,movie,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,2497034.0
1515,the godfather,15400000,No,NaT,NaN,NaN,NaN,tt0068646,movie,1972,\N,175,"Crime,Drama",9.2,1973440.0
1586,gladiator,14900000,No,NaT,NaN,2000,NaN,tt0172495,movie,2000,\N,155,"Action,Adventure,Drama",8.5,1584096.0
2412,the wolf of wall street,9100000,No,NaT,NaN,NaN,NaN,tt0993846,movie,2013,\N,180,"Biography,Comedy,Crime",8.2,1536866.0


In [23]:
# Merge Originial title matching data with already completed data from previous attempts
Complete_data_df = pd.concat([Complete_data_df, Original_title_matching_df], ignore_index=True, sort=False)
Complete_data_df = Complete_data_df.sort_values(by=['Hours Viewed'], ascending=False)
Complete_data_df

,Title,Hours Viewed,Available Globally?,Release Date,originalTitle,Year_Country,release_year,tconst,titleType,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,ginny & georgia,967200000,Yes,2021-02-24,NaN,NaN,2021,tt10813940,tvSeries,2021,\N,\N,"Comedy,Drama",7.5,77525.0
1,the night agent,812100000,Yes,2023-03-23,NaN,NaN,2023,tt13918776,tvSeries,2023,\N,45,"Action,Drama,Thriller",7.5,99387.0
8240,you,766300000,Yes,2018-12-26,NaN,NaN,2018,tt7335184,tvSeries,2018,2024,45,"Crime,Drama,Romance",7.7,289182.0
2,outer banks,740400000,Yes,2020-04-15,NaN,NaN,2020,tt10293938,tvSeries,2020,\N,50,"Action,Crime,Drama",7.5,74515.0
8032,the walking dead,738600000,No,NaT,NaN,NaN,NaN,tt1520211,tvSeries,2010,2022,44,"Drama,Horror,Thriller",8.1,1058747.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6980,the cursed lesson,100000,No,NaT,요가학원,NaN,NaN,tt13478812,movie,2020,\N,93,"Horror,Mystery,Thriller",4.7,211.0
6981,ah boys to men,100000,No,NaT,NaN,NaN,NaN,tt2525596,movie,2012,\N,110,Comedy,6.1,598.0
6982,mix tape,100000,No,NaT,NaN,NaN,NaN,tt0378369,movie,2003,\N,97,Comedy,8.1,15.0
6983,fareast: from berlin to tokyo,100000,No,NaT,fernOST – von Berlin nach Tokio,NaN,NaN,tt3328154,tvSeries,2013,2013,43,Documentary,7.8,73.0


In [24]:
# Export final information to csv file
Complete_data_df.to_csv("Netflix_data.csv", index=False)